In [2]:
import pandas as pd
df = pd.read_csv("filtered.tsv", sep="\t", index_col='Unnamed: 0')
df = df.drop(columns=['similarity', 'lenght_diff'])

In [3]:
cond = df["trn_tox"] > df["ref_tox"]
df.loc[cond, ["reference", "translation"]] = df.loc[cond, ["translation", "reference"]].values
df.loc[cond, ["ref_tox", "trn_tox"]] = df.loc[cond, ["trn_tox", "ref_tox"]].values

In [4]:
df = df[((df["ref_tox"] > 0.8) & (df["trn_tox"] < 0.1))]

In [5]:
df = df[((df['reference'].str.len() > 30) & (df['translation'].str.len() > 20))]

In [6]:
df['reference'].str.len().mean()

65.5854577740249

In [7]:
sorted_df = df.sort_values(by=['ref_tox', 'trn_tox'],
                           key=lambda x: abs(df['ref_tox'] - df['trn_tox']))

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(sorted_df, test_size=0.2, random_state=42, shuffle=False)

In [9]:
from datasets import Dataset, DatasetDict

In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base", device_map="auto")

In [11]:
PREFFIX = "detoxify: "

In [12]:
dataset = Dataset.from_pandas(df.drop(columns=['ref_tox', 'trn_tox']), preserve_index=False)

In [13]:
dataset

Dataset({
    features: ['reference', 'translation'],
    num_rows: 322124
})

In [14]:
train_test_split = dataset.train_test_split(test_size=0.05)

# Take the test part and split it further into validation and test datasets (50-50 split)
validation_test_split = train_test_split['test'].train_test_split(test_size=0.8)

# Now, construct a DatasetDict
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'validation': validation_test_split['train'],
    'test': validation_test_split['test']
})

In [15]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['reference', 'translation'],
        num_rows: 306017
    })
    validation: Dataset({
        features: ['reference', 'translation'],
        num_rows: 3221
    })
    test: Dataset({
        features: ['reference', 'translation'],
        num_rows: 12886
    })
})

In [16]:
def tokenize(examples):
    inputs = [PREFFIX + e for e in examples["reference"]]
    targets = examples["translation"]
    
    tokenized_inputs = tokenizer(inputs, max_length=128, truncation=True)
    tokenized_targets = tokenizer(targets, max_length=128, truncation=True)

    tokenized_inputs["labels"] = tokenized_targets["input_ids"]
    return tokenized_inputs

In [17]:
tokenized_dataset = dataset_dict.map(tokenize, batched=True, remove_columns=['reference', 'translation'])

Map:   0%|          | 0/306017 [00:00<?, ? examples/s]

Map:   0%|          | 0/3221 [00:00<?, ? examples/s]

Map:   0%|          | 0/12886 [00:00<?, ? examples/s]

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 306017
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3221
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 12886
    })
})

In [19]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
import evaluate
import numpy as np

metric = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    print(result)
    result = {"bleu": result["bleu"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [21]:
gconf = model.generation_config

In [22]:
gconf.max_new_tokens = 128
gconf.repetition_penalty = 1.2

In [23]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer



training_args = Seq2SeqTrainingArguments(
    output_dir="t5_detox",
    evaluation_strategy="steps",
    eval_steps=2000,
    logging_strategy="steps",
    logging_steps=200,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    generation_config=gconf,
    report_to='wandb',
)

In [24]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: batalov (contextr). Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Bleu,Gen Len
2000,1.940200,1.741413,0.259700,18.389300
4000,1.850300,1.691743,0.267100,18.365700
6000,1.834200,1.662268,0.269500,18.236600
8000,1.820100,1.643432,0.272100,18.254900
10000,1.756700,1.629483,0.274400,18.198100
12000,1.745000,1.619735,0.274700,18.189700


{'bleu': 0.25969010429550515, 'precisions': [0.5902148878602893, 0.3337349397590361, 0.20875675958097129, 0.13086884591248787], 'brevity_penalty': 0.9588115052909236, 'length_ratio': 0.9596369254538432, 'translation_length': 44721, 'reference_length': 46602}
{'bleu': 0.26709993891652395, 'precisions': [0.5962205076596221, 0.3420735528028149, 0.21597470801870772, 0.13679675910076458], 'brevity_penalty': 0.9586774469196508, 'length_ratio': 0.9595081756147805, 'translation_length': 44715, 'reference_length': 46602}
{'bleu': 0.2694854262314393, 'precisions': [0.6006705368795284, 0.34683777686130857, 0.21976315789473685, 0.13991201587164667], 'brevity_penalty': 0.9525595319999517, 'length_ratio': 0.9536500579374276, 'translation_length': 44442, 'reference_length': 46602}
{'bleu': 0.272086100396631, 'precisions': [0.6027754661388632, 0.34963627546071774, 0.2223625029590468, 0.1417897580320708], 'brevity_penalty': 0.9529864797919304, 'length_ratio': 0.9540577657611261, 'translation_length': 4

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [26]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [38]:
import random

In [40]:
sample = random.sample(range(len(dataset_dict["test"])), k=50)

In [43]:
for_test = [dataset_dict["test"][i] for i in sample]

In [45]:
inps = [e["reference"] for e in for_test]

In [47]:
from tqdm.notebook import tqdm 

In [109]:
gen = []

for inp in tqdm(inps):
    encodeds = tokenizer(PREFFIX + inp, return_tensors="pt")
    model_inputs = encodeds.to('cuda')
    generated_ids = model.generate(**model_inputs, max_new_tokens=128)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    gen.append(decoded[0])

  0%|          | 0/50 [00:00<?, ?it/s]

In [110]:
gen

['If anyone else is, you should think you screwed up the Rockettes.',
 '"We should have stuffed him in the hole behind his friends," said Dextrinos.',
 "Uh, yeah, I'm really sorry about that.",
 "and I'm going crazy because I don't.",
 'Stephen Morton shot and killed 15 people.',
 "why didn't you just cut me out when you had the chance?",
 "No, we shouldn't tell her because she's going to go crazy.",
 'what are they gonna do?',
 'The black men shot at me as a target, and the dogs tried to tear me apart.',
 "What's the matter with you?",
 "you won't throw it in the trash can when you leave?",
 "but he's got Red's money.",
 "I'm still bringing her to jail, okay?",
 "we moved two savages into the adjacent room, and now we can't sleep.",
 'who cares about being friends with Jane and Brad, if we can be friends with Max and Grant?',
 'who asked you anything?',
 "But if she didn't want it, it would be silly.",
 'you make me look crazy in front of my family!',
 "Only white people don't go to K

In [112]:
# removed_indexes = []
# new_gen = []

# for i, e in enumerate(gen):
#     q = ''.join([s for s in e if s.isalpha() or s == ' '])
#     q = q.strip()
#     if len(q) < 5:
#         removed_indexes.append(i)
#     else:
#         new_gen.append(q)

In [113]:
new_gen = gen

In [114]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification


r_tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
r_model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [115]:
import numpy as np

In [116]:


def classify_preds_toxicity(preds, batch_size=16):
    results = []

    for i in tqdm(range(0, len(preds), batch_size)):
        batch = r_tokenizer(preds[i:i + batch_size], return_tensors='pt', padding=True)
        result = (r_model(**batch)['logits'] / 2.5).softmax(dim=1)[:,1].data.tolist()
        results.extend([1 - item for item in result])

    return np.array(results)

In [117]:
non_toxixty = classify_preds_toxicity(new_gen)

  0%|          | 0/4 [00:00<?, ?it/s]

In [118]:
from nltk.translate.bleu_score import sentence_bleu

def calc_bleu(inputs, preds):
    results = []
    
    print('Calculating BLEU similarity')
    for i in range(len(inputs)):
        results.append(sentence_bleu([inputs[i]], preds[i]))

    return np.array(results)

In [119]:
bleu = calc_bleu(inps, new_gen)

Calculating BLEU similarity


In [120]:
from flair.data import Sentence
from flair.embeddings import FlairEmbeddings
from torch.nn.functional import cosine_similarity
import torch

In [121]:
def flair_sim(inputs, preds, batch_size = 16):
    print('Calculating flair embeddings similarity')
    sim = 0
    batch_size = 16
    inp_embed = []
    pred_embed = []

    embedder = FlairEmbeddings('news-forward')

    for i in range(0, len(inputs), batch_size):
        inp_part = [Sentence(sent) for sent in inputs[i:i + batch_size]]
        pred_part = [Sentence(sent) for sent in preds[i:i + batch_size]]

        inp_part = embedder.embed(inp_part)
        pred_part = embedder.embed(pred_part)

        for j in range(batch_size):
            if ((i + j) < len(inputs)):
                inp_sent_vec = torch.zeros(2048).cuda()
                pred_sent_vec = torch.zeros(2048).cuda()

                for k in range(len(inp_part[j])):
                    inp_sent_vec += inp_part[j][k].embedding
                inp_embed.append(inp_sent_vec.cpu() / (k + 1))

                for k in range(len(pred_part[j])):
                    pred_sent_vec += pred_part[j][k].embedding
                pred_embed.append(pred_sent_vec.cpu() / (k + 1))

    emb_sim = cosine_similarity(torch.stack(inp_embed), torch.stack(pred_embed))

    return emb_sim.numpy()

In [122]:
emb_sim = flair_sim(inps, new_gen)

Calculating flair embeddings similarity


In [123]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

cola_tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-CoLA")
cola_model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-CoLA")


def classify_cola(preds, batch_size=16):
    results = []

    for i in tqdm(range(0, len(preds), batch_size)):
        batch = cola_tokenizer(preds[i:i + batch_size], return_tensors='pt', padding=True)
        result = (cola_model(**batch)['logits']).softmax(dim=1)[:,1].data.tolist()
        results.extend(result)

    return np.array(results)

Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [124]:
cola = classify_cola(new_gen)

  0%|          | 0/4 [00:00<?, ?it/s]

In [125]:
final_score = non_toxixty * np.mean([bleu, emb_sim], axis=0) * cola

In [126]:
final_score.sum() / 50

0.45200450512875245